In [1]:
import time
import random

# ==========================================
# CORE DATA STRUCTURES (The "Architect" Work)
# ==========================================

class Node:
    """
    A Node in the Doubly Linked List.
    Holds the Key and Value, plus pointers to Prev and Next nodes.
    """
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:
    """
    The main Cache class using Hash Map + Doubly Linked List.
    Time Complexity: O(1) for both get and put.
    """
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  # Hash Map (Python Dictionary) mapping key -> Node
        
        # Initialize Dummy Head and Dummy Tail
        # These make adding/removing nodes easier (no null checks needed)
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head

    def _remove(self, node):
        """Helper: Removes a node from the Linked List."""
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node

    def _add(self, node):
        """Helper: Adds a node right after the Head (Most Recently Used position)."""
        prev_node = self.head
        next_node = self.head.next
        
        prev_node.next = node
        node.prev = prev_node
        node.next = next_node
        next_node.prev = node

    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            # 1. Remove from current position
            self._remove(node)
            # 2. Add to front (Head)
            self._add(node)
            return node.value
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # Update existing node
            self._remove(self.cache[key])
        
        new_node = Node(key, value)
        self._add(new_node)
        self.cache[key] = new_node
        
        if len(self.cache) > self.capacity:
            # Evict the Least Recently Used (Node before Tail)
            lru_node = self.tail.prev
            self._remove(lru_node)
            del self.cache[lru_node.key]

    # ==========================================
    # VISUALIZATION TOOL (The "Visualizer" Work)
    # ==========================================
    def display(self):
        """Prints the current state of the cache visually."""
        elements = []
        curr = self.head.next
        while curr != self.tail:
            elements.append(f"[{curr.key}:{curr.value}]")
            curr = curr.next
        
        print(f"Hash Map Size: {len(self.cache)}/{self.capacity}")
        print("Order (MRU -> LRU): HEAD <-> " + " <-> ".join(elements) + " <-> TAIL")
        print("-" * 60)

# ==========================================
# DEMONSTRATION SCRIPT
# ==========================================
if __name__ == "__main__":
    print("=== INITIALIZING LRU CACHE (Capacity 3) ===\n")
    lru = LRUCache(3)
    lru.display()

    print(">>> 1. Put (1, 'A')")
    lru.put(1, 'A')
    lru.display()

    print(">>> 2. Put (2, 'B')")
    lru.put(2, 'B')
    lru.display()

    print(">>> 3. Put (3, 'C') - Cache is now FULL")
    lru.put(3, 'C')
    lru.display()

    print(">>> 4. Get (1) - Key 1 was used, so it moves to HEAD (MRU)")
    lru.get(1)
    lru.display()

    print(">>> 5. Put (4, 'D') - Capacity Exceeded! Evicting LRU (Key 2)")
    lru.put(4, 'D')
    lru.display()

=== INITIALIZING LRU CACHE (Capacity 3) ===

Hash Map Size: 0/3
Order (MRU -> LRU): HEAD <->  <-> TAIL
------------------------------------------------------------
>>> 1. Put (1, 'A')
Hash Map Size: 1/3
Order (MRU -> LRU): HEAD <-> [1:A] <-> TAIL
------------------------------------------------------------
>>> 2. Put (2, 'B')
Hash Map Size: 2/3
Order (MRU -> LRU): HEAD <-> [2:B] <-> [1:A] <-> TAIL
------------------------------------------------------------
>>> 3. Put (3, 'C') - Cache is now FULL
Hash Map Size: 3/3
Order (MRU -> LRU): HEAD <-> [3:C] <-> [2:B] <-> [1:A] <-> TAIL
------------------------------------------------------------
>>> 4. Get (1) - Key 1 was used, so it moves to HEAD (MRU)
Hash Map Size: 3/3
Order (MRU -> LRU): HEAD <-> [1:A] <-> [3:C] <-> [2:B] <-> TAIL
------------------------------------------------------------
>>> 5. Put (4, 'D') - Capacity Exceeded! Evicting LRU (Key 2)
Hash Map Size: 3/3
Order (MRU -> LRU): HEAD <-> [4:D] <-> [1:A] <-> [3:C] <-> TAIL
------